In [61]:
from dbanalysis import stop_tools

## Trying some different models on a single stop

In [62]:
df = stop_tools.random_stop_data()

In [63]:
from sklearn.ensemble import RandomForestRegressor as rf

In [64]:
df.columns

Index(['index', 'dayofservice', 'tripid', 'plannedtime_arr_from',
       'plannedtime_dep_from', 'actualtime_arr_from', 'actualtime_dep_from',
       'plannedtime_arr_to', 'actualtime_arr_to', 'routeid', 'fromstop',
       'tostop', 'traveltime', 'dwelltime', 'distance', 'speed', 'dt', 'date',
       'day', 'month', 'hour', 'year', 'dewpt', 'msl', 'rain', 'rhum', 'temp',
       'vappr', 'wetb'],
      dtype='object')

In [65]:
df['month']=df['dt'].dt.month
train = df[df['month']<5]
test=df[df['month']>=5]

In [66]:
train.shape

(15526, 29)

In [67]:
test.shape

(8107, 29)

In [68]:
train = train.dropna()

In [69]:
train.columns

Index(['index', 'dayofservice', 'tripid', 'plannedtime_arr_from',
       'plannedtime_dep_from', 'actualtime_arr_from', 'actualtime_dep_from',
       'plannedtime_arr_to', 'actualtime_arr_to', 'routeid', 'fromstop',
       'tostop', 'traveltime', 'dwelltime', 'distance', 'speed', 'dt', 'date',
       'day', 'month', 'hour', 'year', 'dewpt', 'msl', 'rain', 'rhum', 'temp',
       'vappr', 'wetb'],
      dtype='object')

In [70]:
df['day']=df['dt'].dt.dayofweek

In [71]:
regr = rf(max_depth=100).fit(train[['month','hour','day','actualtime_arr_from','actualtime_dep_from','rain','rhum','temp','msl','wetb']],train['traveltime'])

In [72]:
preds = regr.predict(test[['month','hour','day','actualtime_arr_from','actualtime_dep_from','rain','rhum','temp','msl','wetb']])

In [73]:
from sklearn import metrics

In [74]:
metrics.mean_absolute_error(test['traveltime'],preds)

20.008918218823236

In [75]:
metrics.r2_score(test['traveltime'],preds)

-0.21138020631965593

In [76]:
import numpy as np
msk = np.random.rand(len(df))<0.8

In [77]:
df=df.dropna()
train = df[msk]
test = df[~msk]

In [78]:
regr = rf(max_depth=100).fit(train[['month','hour','day','actualtime_arr_from','actualtime_dep_from','rain','rhum','temp','msl','wetb']],train['traveltime'])
preds = regr.predict(test[['month','hour','day','actualtime_arr_from','actualtime_dep_from','rain','rhum','temp','msl','wetb']])
metrics.mean_absolute_error(test['traveltime'],preds)

18.767603833865813

In [79]:
df['act2']=df['actualtime_arr_from']**2
df['act3']=df['actualtime_arr_from']**3
df['rain2']=df['rain']**2
df['rain3']=df['rain']**3
df['day2']=df['day']**2
df['day3']=df['day']**3

In [80]:
features=['month','hour','day','actualtime_arr_from','actualtime_dep_from','rain','rhum','temp','msl','wetb','act2','act3','rain2','rain3','day2','day3']

In [81]:
regr = rf(max_depth=100).fit(train[features],train['traveltime'])
preds = regr.predict(test[features])
metrics.mean_absolute_error(test['traveltime'],preds)

KeyError: "['act2' 'act3' 'rain2' 'rain3' 'day2' 'day3'] not in index"

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
regr = LinearRegression(fit_intercept=True).fit(train[features],train['traveltime'])
preds=regr.predict(test[features])
metrics.mean_absolute_error(test['traveltime'],preds)

In [ ]:
from sklearn.neural_network import MLPRegressor as ml

In [ ]:
regr = ml().fit(train[features],train['actualtime_arr_to'])

In [ ]:
preds=regr.predict(test[features])

In [ ]:
metrics.mean_absolute_error(test['actualtime_arr_to'],preds)

In [19]:
features=['month',
 'hour',
 'day',
 'actualtime_arr_from',
 'actualtime_dep_from',
 'rain',
 'rhum',
 'temp',
 'msl',
 'wetb']

In [ ]:
from sklearn import svm

In [ ]:
regr =svm.SVR().fit(train[features],train['traveltime'])

In [ ]:
preds=regr.predict(test[features])

In [ ]:
import pandas as pd

In [ ]:
#dumb method
preds = pd.Series([test['traveltime'].mean() for i in range(len(test))])

In [ ]:
metrics.mean_absolute_error(test['traveltime'],preds)

### The point here then is that predicting the mean isn't really so bad

In [1]:
import json
routes = json.loads(open('/home/student/dbanalysis/dbanalysis/resources/trimmed_routes.json','r').read())

eh fuck this


# Making a singular gigantic model of a route

In [2]:
r=routes['15'][1]

In [3]:
from dbanalysis import stop_tools
import pandas as pd

In [4]:
to_concat=[]
for i in range(1, len(r)-1):
    a=r[i]
    b=r[i+1]
    df=stop_tools.get_stop_link(a,b,merge_weather=True)
    to_concat.append(df)
    
df = pd.concat(to_concat,axis=0)
    

In [5]:
df=df.dropna()

In [ ]:
df.shape

## adding the approx distances from the first stop

In [6]:
s_getter =stop_tools.stop_getter()
total_distance = 0
route_distances = {r[1]:0}
for i in range(1, len(r)-1):
    distance = s_getter.get_stop_distance(str(r[i]),str(r[i+1]))
    
    total_distance += distance
    route_distances[r[i+1]]=total_distance
    

In [ ]:
route_distances

In [7]:
df['distance']=df['stopA'].apply(lambda x: route_distances[x])

## hacking in the time of leaving the first stop

In [8]:
keys= df[df['stopA']==6318]

In [ ]:
len(keys)

In [9]:
to_concat = []
## no this doesn't work
#for row in keys.itertuples():

  #  
 #   temp_df = df[(df['dayofservice']==row[1]) & (df['tripid']==row[2])]
   # temp_df['base_time_dep'] = row[5]
#    to_concat.append(temp_df)


keys['base_time_dep']=keys['actualtime_arr_from']
keys2=keys[['tripid','dayofservice','base_time_dep']]
full_df = pd.merge(df,keys2,on=['dayofservice','tripid'])
    

/home/student/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
full_df

In [12]:
full_df['base_time_dep'].max()

84703

In [ ]:
full_df.shape

In [ ]:
# we seem to have lost a lot of samples unfortunately, by virtue of following this process. Maybe we need to like ... use the scheduled departure times instead?

In [ ]:
full_df.columns

In [10]:
full_df['traveltime']=full_df['actualtime_arr_from']-full_df['base_time_dep']

In [ ]:
full_df

## Splitting and adding date time information

In [17]:
import numpy as np
msk = np.random.rand(len(full_df))<0.8
test = full_df[msk]

train = full_df[~msk]

In [11]:
full_df=full_df.dropna()

In [14]:
time_format = "%d-%b-%y %H:%M:%S"
full_df['dt']=pd.to_datetime(full_df['dayofservice'],format=time_format)
full_df['dayofweek']=full_df['dt'].dt.dayofweek
full_df['month']=full_df['dt'].dt.month
full_df['weekend']=full_df['dayofweek']>4
full_df['year']=full_df['dt'].dt.year

In [16]:
train=full_df[full_df['year']==2016]
test = full_df[full_df['year']==2017]

In [18]:
from sklearn.linear_model import LinearRegression
features=['hour','weekend','month','dayofweek','rhum','rain','vappr','wetb','distance','base_time_dep','msl','temp']
regr=LinearRegression(fit_intercept=True).fit(train[features],train['actualtime_arr_from'])

In [20]:
from sklearn import metrics
predicts=regr.predict(test[features])
print(metrics.mean_absolute_error(test['actualtime_arr_from'],predicts))
print(metrics.r2_score(test['actualtime_arr_from'],predicts))

321.6330604006038
0.999340977841599


In [21]:
full_df['weekend']=full_df['dayofweek']>4
features=['hour','weekend','month','dayofweek','rhum','rain','vappr','wetb','distance','base_time_dep','msl','temp']
regr=LinearRegression(fit_intercept=True).fit(train[features],train['traveltime'])

In [22]:
predicts=regr.predict(test[features])
print(metrics.mean_absolute_error(test['traveltime'],predicts))
print(metrics.r2_score(test['traveltime'],predicts))

321.6330604005971
0.9280429215025939


# Model works, but there is clearly something bullshit about these r2 scores

... Testing again, but this time only on a single variable (the time taken to get from the first stop to the last stop)

In [23]:
test2 = full_df[full_df['distance']==22.60855934258725]

In [68]:
len(test2)

6593

In [24]:
predicts=regr.predict(test2[features])
print(metrics.mean_absolute_error(test2['traveltime'],predicts))
print(metrics.r2_score(test2['traveltime'],predicts))

534.2927397275263
0.22018558098111973


## As you can see

# a) The r2 score has collapsed now that we're testing only a singular variable

# b) The mean absolute error has gone up - we are about 10 minutes off in our predictions for how long it takes the bus to travel the entirety of the route

## Over all, I'm going to say that the only gains from making 'whole route models' are going to be in terms of optimizing our compute process, not in terms of accuracy.

## The two step dwelltime/traveltime modelling process still makes way more sense I reckon

With this gigantic route model approach we're:
1. Forced to drop a lot of relevant data from our analysis (the links that are shared by multiple routes)
2. Trying to approximate a horrendously complicated, messy, non linear function.
3. Unable to adequately separate the prediction of dwell time and the prediction of travel time.





# With fake neural network

In [ ]:
regr = ml().fit(train[features],train['traveltime'])

In [ ]:
predicts=regr.predict(test2[features])
print(metrics.mean_absolute_error(test2['traveltime'],predicts))
print(metrics.r2_score(test2['traveltime'],predicts))

## It also takes a shameful amount of time to train a neural network model on this ... 


### And we only save on 5 seconds

# With  svm

In [37]:
from sklearn import svm

In [28]:
regr = svm.SVR().fit(train[features],train['traveltime'])


In [29]:
import pickle
with open('svm.pickle','wb') as handle:
    pickle.dump(regr,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [38]:
import pickle
with open('svm.pickle','rb') as handle:
    regr = pickle.load(handle)

In [ ]:
from sklearn import metrics
print('### All stops ###')
predicts=regr.predict(test[features])
print(metrics.mean_absolute_error(test['traveltime'],predicts))
print(metrics.r2_score(test['traveltime'],predicts))
predicts=regr.predict(test2[features])
print('\n\n ### One Stop ###')
print(metrics.mean_absolute_error(test2['traveltime'],predicts))
print(metrics.r2_score(test2['traveltime'],predicts))

### All stops ###


In [54]:
predicts=regr.predict(test2[features])
from sklearn import metrics
print(metrics.mean_absolute_error(test2['traveltime'],predicts))
print(metrics.r2_score(test2['traveltime'],predicts))

2591.2068976663527
-11.535763697371335


# Forget about training time - svm seems to take forever to predict?

Also the results are crap. Clearly will haven to understand what this model actually does, before fine tuning it.

## With random forest

In [22]:
features=['hour','weekend','month','dayofweek','rhum','rain','vappr','wetb','distance','base_time_dep','msl','temp']

In [17]:
from sklearn.ensemble import RandomForestRegressor as rf

In [20]:
features

['month',
 'hour',
 'day',
 'actualtime_arr_from',
 'actualtime_dep_from',
 'rain',
 'rhum',
 'temp',
 'msl',
 'wetb']

In [29]:
test['distance'].unique()

array([ 0.        ,  0.19439372,  0.42661594,  0.91399099,  2.12746783,
        2.13710899,  2.73836243,  3.05509448,  3.47239308,  3.98918733,
        4.29713731,  4.60077595,  5.07080026,  5.22224552,  5.52456992,
        5.73902979,  6.13097876,  6.47090175,  6.80926407,  7.12874632,
        7.41718128,  7.73113298,  8.15250246,  8.459664  ,  8.79233752,
        9.06522538,  9.34944473,  9.63199619,  9.91663643, 10.38606964,
       11.79891293, 12.14116219, 12.56124053, 13.38822997, 13.59259852,
       13.8187145 , 13.9437401 , 14.33519184, 14.54172718, 14.83736879,
       15.17371397, 15.38787347, 15.67475222, 15.87786843, 16.06248638,
       16.37022232, 16.61976575, 16.90819453, 17.16837317, 17.39439661,
       17.86033059, 18.30079427, 19.16280228, 19.600695  , 20.9327889 ,
       21.35775595, 22.28254239,  2.41355696, 10.07682693, 14.10590383,
       18.76220611, 19.86110042, 20.63406303, 22.17246364, 22.60855934,
       20.46557701, 13.00315193, 11.24524183, 11.02329844])

In [23]:
regr = rf().fit(train[features],train['traveltime'])

In [24]:
test2 = test[test['distance']==test['distance'].max()]
predicts=regr.predict(test2[features])

from sklearn import metrics
print(metrics.mean_absolute_error(test2['traveltime'],predicts))
print(metrics.r2_score(test2['traveltime'],predicts))

378.17156074624603
0.5809780256092053


In [26]:
import sys
sys.getsizeof(regr)

56

In [29]:
from sklearn.tools import joblib

ModuleNotFoundError: No module named 'sklearn.tools'

In [ ]:
import pickle
with open('/data/forestdump.pickle','wb') as handle:
    pickle.dump(regr,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
((abs(test2['traveltime']-predicts)/test2['traveltime'])*100).mean()

7.816852303727892

In [97]:
predicts=regr.predict(test2[features])

from sklearn import metrics
print(metrics.mean_absolute_error(test2['traveltime'],predicts))
print(metrics.r2_score(test2['traveltime'],predicts))

269.99855513307983
0.7703147250617991


In [42]:
test[(test['distance']>2.537109) & (test['distance']<3)]['distance'].unique()

array([2.73836243])

# This seems to be the best candidate so far

Large reduction in error from what we saw while using linear and neural networks.

Why not cluster the data then to find the best candidates for this kind of thing? Somehow minimize the size of the models through sampling important data or something. Yeah. That kind of thing.


# With k nearest neighbors

In [76]:
from sklearn.neighbors import KNeighborsRegressor

In [87]:
regr = KNeighborsRegressor(1).fit(train[features],train['traveltime'])
predicts=regr.predict(test2[features])
print(metrics.mean_absolute_error(test2['traveltime'],predicts))
print(metrics.r2_score(test2['traveltime'],predicts))

312.9153992395437
0.6483993230964855


#### Interestingly, upping the number of neighbors seems to push the results into negative territory

In [ ]:
print('hello')

# All that said, the biggest problem with the big route model is that we're dropping so much data

### How to use the rows of data that come from different routes? Is there a legitimate way to use them even?





# Bascially have to do this notebook again with a 2016/2017 split

In [57]:
features

['hour',
 'weekend',
 'month',
 'dayofweek',
 'rhum',
 'rain',
 'vappr',
 'wetb',
 'distance',
 'base_time_dep',
 'msl',
 'temp']